In [1]:
import numpy as np 
import pandas as pd 
from sklearn.metrics import f1_score, accuracy_score

In [ ]:
train = pd.read_pickle('./data/train_cutted_tokenized.pkl')
train.drop('text', axis=1, inplace=True)

val = pd.read_pickle('./data/val_cutted_tokenized.pkl')
val.drop('text', axis=1, inplace=True)

train

In [3]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
fp16 = True if torch.cuda.is_available() else False

label_columns = ['pornographic-content', 'violence', 'death', 'sexual-assault', 'abuse', 'blood',
                 'suicide', 'pregnancy', 'child-abuse', 'incest', 'underage', 'homophobia',
                 'self-harm', 'dying', 'kidnapping', 'mental-illness', 'dissection',
                 'eating-disorders', 'abduction', 'body-hatred', 'childbirth', 'racism',
                 'sexism', 'miscarriages', 'transphobia', 'abortion', 'fat-phobia',
                 'animal-death', 'ableism', 'classism', 'misogyny', 'animal-cruelty']

X_train = torch.tensor(train["tokens"]).to(device)
train_masks = torch.tensor(train["mask"]).to(device)
y_train = torch.tensor(train[label_columns].values, dtype=torch.float32).to(device)

X_val = torch.tensor(val["tokens"]).to(device)
val_masks = torch.tensor(val["mask"]).to(device)
y_val = torch.tensor(val[label_columns].values, dtype=torch.float32).to(device)

del train, val

cuda


C:\Users\Anwender\AppData\Local\Temp\ipykernel_14160\2426158358.py:14: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  train_masks = torch.tensor(train["mask"]).to(device)


In [4]:
from transformers import AdamW, BigBirdTokenizer ,get_linear_schedule_with_warmup, BigBirdForSequenceClassification, Trainer, TrainingArguments,EvalPrediction, AutoTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader

model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                gradient_checkpointing=False,
                num_labels=32,
                problem_type="multi_label_classification",
                return_dict=True)

model.to(device)

C:\Users\Anwender\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [5]:
from util_loss import ResampleLoss
from collections import Counter
np.set_printoptions(suppress=True)

train_num = len(X_train)
print(train_num)
class_freq = y_train.sum(axis=0).cpu().numpy().astype(int)
print(class_freq)

loss_func = ResampleLoss(reweight_func='rebalance', loss_weight=1.0,
                         focal=dict(focal=True, alpha=0.5, gamma=2),
                         logit_reg=dict(init_bias=0.05, neg_scale=2.0),
                         map_param=dict(alpha=0.1, beta=10.0, gamma=0.9), 
                         class_freq=class_freq, train_num=train_num)
loss_func

307102
[238075  29114  20785  31320  22186  15099   8208  13649   7183  13492
   8904   4945   5241   7498   4482   4171   1708   1209   1060   1365
    854    399    533    507    377    344    738    208    267    187
    222    150]


ResampleLoss()

In [ ]:
from tqdm import tqdm
import os
from torch.cuda.amp import autocast, GradScaler
from sklearn.utils import class_weight

#batch_size = 16
epochs = 2
#learning_rate = 2e-5

#criterion = torch.nn.BCEWithLogitsLoss()

# Convert your PyTorch tensors to DataLoader
#train_dataset = torch.utils.data.TensorDataset(X_train, train_masks, y_train)
#val_dataset = torch.utils.data.TensorDataset(X_val, val_masks, y_val)
#train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#val_loader = DataLoader(val_dataset, batch_size=batch_size)

#model.load_state_dict(torch.load('saved_models/model_epoch_3.pt')) # load model if wanted

#optimizer = AdamW(model.parameters(), lr=learning_rate)
#scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * epochs)

#save_dir = "saved_models_db_loss"
#os.makedirs(save_dir, exist_ok=True)

#scaler = GradScaler()

### code for saving
def save_model(model, epoch):
    model_save_path = os.path.join(save_dir, f"model_epoch_{epoch+5}.pt")
    torch.save(model.state_dict(), model_save_path)

### code to evaluate model
def evaluate(model, val_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in val_loader:
            input_ids, attention_mask, labels = batch
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = (logits > 0).int()  # Convert logits to binary predictions
            all_preds.extend(preds.tolist())
            all_labels.extend(labels.tolist())
    accuracy = accuracy_score(all_labels, all_preds)
    f1_micro = f1_score(all_labels, all_preds, average='micro')
    f1_macro = f1_score(all_labels, all_preds, average='macro')
    acc = accuracy_score(all_labels, all_preds)
    return acc, f1_micro, f1_macro

for epoch in range(epochs):
    train_loss = 0.0
    model.train()
    with tqdm(total=len(train_loader), desc=f"Epoch {epoch + 1}/{epochs}", unit="batch") as pbar:
        for batch in train_loader:
            input_ids, attention_mask, labels = batch
            optimizer.zero_grad()

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

            loss = loss_func(outputs.logits.view(-1, 32), labels.type_as(outputs.logits).view(-1, 32))
            
            #loss = criterion(outputs.logits, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            train_loss += loss.item()
            pbar.update(1)  # Update the progress bar
    train_loss /= len(train_loader)
    
    acc, f1_micro, f1_macr = evaluate(model, val_loader)

    save_model(model, epoch + 1)
    
    print(f"Epoch {epoch + 1}/{epochs}")
    print(f"Train Loss: {train_loss:.4f}")
    print(f"Validation Accuracy: {acc:.4f}")
    print(f"Validation F1 Micro: {f1_micro:.4f}")
    print(f"Validation F1 Macro: {f1_macr:.4f}")
    print("---------------------------")


Epoch 1/2: 100%|████████████████████████████████████████████████████████████| 19194/19194 [3:41:47<00:00,  1.44batch/s]


Epoch 1/2
Train Loss: 0.0014
Validation Accuracy: 0.5950
Validation F1 Micro: 0.7346
Validation F1 Macro: 0.3566
---------------------------


Epoch 2/2:   3%|█▊                                                            | 571/19194 [06:31<3:34:45,  1.45batch/s]